# 실습 목표  

main.ipynb를 작성하여 eval dataset에 대해서 PESQ 1.3 이상 달성  


# 시작하기   
**_[Run]_** 항목은 추가 구현 없이 실행      
**_[Run]_** 항목은 구현 하여 실행  
**_[Option]_** 항목은 필수적으로 구현할 필요는 없으나 필요시 구현  


### clean 데이터
+ [AI HUB 한국어 음성 데이터](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=123)  
2명의 화자가 특정 주제에 대해서 자유롭게 발화하는 데이터셋  

### noise 데이터
+ [CHiME4 background noises](https://spandh.dcs.shef.ac.uk/chime_challenge/CHiME4/data.html#Backgrounds)의 STR  
특정 장소(STR : 길가)에서 장시간 녹음한 잡음 데이터  


---

# Data & Code download  

## 다운받을 데이터 구성  
+ train    
    매 학습시 새로 합성하는 on-fly로 진행하기 때문에 clean, noise 파일로 구성
+ dev  
    train,eval과 겹치지 않는 데이터로 합성된 noisy 데이터  
+ eval  
    dev,eval과 겹치지 않는 데이터로 합성된 noisy 데이터
+ src :   
    사용될 코드들  

**_[Run]구글 드라이브에서 실습자료 다운로드_**     

In [8]:
!pip install pesq

from os.path import exists

if not exists("SE_dataset.tar") :
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='1hSqrUQWbwv-JmWvSbhsziA6-BwW5wFkV -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="1hSqrUQWbwv-JmWvSbhsziA6-BwW5wFkV -O SE_dataset.tar && rm -rf /tmp/cookies.txt
    !tar -xf SE_dataset.tar
else : 
    print("Dataset already downloaded.")

Dataset already downloaded.


# Init

https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html

In [ ]:
%load_ext autoreload
%autoreload 2

**_[RUN] 모듈 불러오기_**

In [ ]:
import torch
import torch.nn as nn
from src.dataset import DatasetMix,DatasetFix
from src.UNet import UNet
from src.utils import get_output_wav,train,infer

import matplotlib.pyplot as plt 
import IPython.display as ipd

# Dataset 

DatasetMix :   
개별 clean wav에서 1초 길이를 샘플링, 하나의 긴 noise 파일에서 1초 길이를 샘플링 한 뒤  
SNR 0dB ~ 10dB 로 랜덤하게 합성   

DatasetFix :  
미리 합성된 오디오를 사용  

**_[RUN] Dataset 생성_**

In [ ]:
dataset_train = DatasetMix("clean_train","noise_train.wav")
dataset_dev = DatasetFix("dev")
dataset_eval = DatasetFix("eval")

# Model Implementation

## UNet

![image.png](https://drive.google.com/uc?id=11liiywXpjgAn96o6ccDLZVO5QtCzBkBX)  

위 그림은 참고용 UNet 구조 예시이다.  
+ UNet은 Encoder를 통해 입력을 압축하고 Decoder를 통해 원래 차원으로 복원한다.     
+ 각 층에서의 Encoder 출력을 대응되는 층의 Decoder 단에서 concat되어서 Decoder의 입력으로 사용된다.   
  => Encoder의 출력의 크기는 대응 되는 Decoder 층의 이전 Decoder의 출력과 같아야한다.  
+ concat되는 데이터는 Residual path를 사용하여 레이어를 거칠 수도 있고, skip connection으로 아무런 조작없이 사용할 수도 있다.    
+ Encoder의 끝과 Decoder의 시작 사이에 bottleneck 레이러를 두기도 한다.  

### Input  

+ D. S. Williamson, Y. Wang and D. Wang, "Complex Ratio Masking for Monaural Speech Separation," in IEEE/ACM Transactions on Audio, Speech, and Language Processing, vol. 24, no. 3, pp. 483-492, March 2016, doi: 10.1109/TASLP.2015.2512042.  
![image.png](https://drive.google.com/uc?id=1jWyLRn4WqKVi7PqtFIVzf4o1kbJJ3tIE)   

noisy를 STFT 취한뒤, complex domain에서 magnitude를 입력으로 사용한다.
phase 성분은 위 그림처럼 제대로 정보를 취득하기 어렵기 때문에 본 실습에서는 사용하지 않는다.  


### Output  

magnitude의 Mask를 모델의 출력으로 사용한다. 입력의 magnitude에 생성된 마스크를 사용하여 잡음을 제거하고  
입력의 phase를 사용하여 WAV 신호로 복원한다.  


**_[TODO]Encoder,Decoder 구현_**  

## Encoder


![image.png](https://drive.google.com/uc?id=1yhiWPvBizH4dVdnHz5MZkv5XEH4xPiVq)  
Conv2d(Convolution) <-> ConvTranspose2d(Deconvolution)  

기본적인 Encoder,Decoder 모듈은 제공  
encoder,decoder list 에 같은 갯수의 Encoder,Decoder를 쌓아서 UNet을 구성  
+ 모듈 쌓기   
+ 모듈 수정하기
+ 모듈ㅇ

In [7]:
class Encoder(nn.Module):
    def __init__(self, 
                 in_channels, 
                 out_channels, 
                 kernel_size, 
                 stride, 
                 padding):
        super().__init__()
      
        self.conv = nn.Conv2d(in_channels, 
                              out_channels, 
                              kernel_size=kernel_size, 
                              stride=stride,
                              padding=padding)
        self.norm = nn.BatchNorm2d(out_channels)
        self.acti = nn.LeakyReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.acti(x)
        return x

encoders=[]
print(len(encoders))

## Decoder  

In [ ]:
class Decoder(nn.Module):
    def __init__(self, 
                 in_channels, 
                 out_channels,
                 kernel_size, 
                 stride, 
                 output_padding,
                 padding=(0, 0)):
        super().__init__()
       
        self.transconv = nn.ConvTranspose2d(
            in_channels, 
            out_channels, 
            kernel_size=kernel_size,
            stride=stride, 
            output_padding=output_padding,
            padding=padding)
        self.norm = nn.BatchNorm2d(out_channels)
        self.acti = nn.LeakyReLU(inplace=True)

    def forward(self, x):
        x = self.transconv(x)
        x = self.norm(x)
        x = self.acti(x)
        return x
    
decoders=[]
print(len(decoders))

**_[Option]Residual Path,bottleneck 구현_**

## Optional

### Bottleneck

### Residual Path

## UNet

**_[RUN]모델 정상 동작 확인_**  
: 에러가 발생하지 않아야한다. 

In [ ]:
model = UNet(encoders,decoders)
x = torch.rand(1,1,257,126)
y = model(x)
print(y.shape)

# SDR(Signal-to-Distortion Ratio) Loss

+ Venkataramani, Shrikant, Jonah Casebeer, and Paris Smaragdis. "Adaptive front-ends for end-to-end source separation." Proc. NIPS. 2017.  
+ Choi, Hyeong-Seok, et al. "Phase-aware speech enhancement with deep complex u-net." International Conference on Learning Representations. 2018.  

![image.png](https://drive.google.com/uc?id=1bis4FlepY8H9TULmapLEX7XU9QcL5Iqi)


## TODO  

SDR에 관한 수학적 이야기 



**_[TODO]mSDRLoss 구현_**     
: (6) 식 구현

+ 참고    
  TODO : 몇가지 torch linalg 함수들 .. 

In [4]:
"""
    x     : noisy = y+z
    y     : clean
    y_hat : estimated clean 
    z     : noise
    z_hat : estimated noise
"""

def mSDRLoss(y,y_hat, eps=1e-7):
    ## mSDRLoss 구현
    
    return mSDR

def wSDRLoss(y,x,y_hat,alpha=0.1,eps=1e-7):
        z = x - y
        z_hat = x - y_hat

        wSDR = alpha * mSDRLoss(y,y_hat,eps=eps) + (1-alpha)*mSDRLoss(z,z_hat,eps=eps)
        return wSDR
    

# Train

**_[TODO]학습 진행_**

In [5]:
train(model,dataset_train,dataset_dev,wSDRLoss)

## Visualization  
**_[Run]모델 출력 시각화_**

In [ ]:
# Data to visualize
idx = 0

model.eval()
with torch.no_grad():
    clean,noisy,estim = infer(dataset_eval[idx],model)
    
    ipd.display(ipd.Markdown('+ clean'))
    display(ipd.Audio(clean,rate=16000))
    
    plot_spec(clean,"clean")
    plot_spec(noisy,"noisy")
    plot_spec(estim,"estim")
    
    ipd.display(ipd.Markdown('+ noisy'))
    display(ipd.Audio(noisy,rate=16000))
    
    ipd.display(ipd.Markdown('+ estim'))
    display(ipd.Audio(estim,rate=16000))

# Evaluation

PESQ 설명  

**_[TODO] PESQ 1.3 이상 달성_**

In [ ]:
from src.utils import infer,plot_spec

from pesq import pesq
from pesq import NoUtterancesError                         

model.eval()
with torch.no_grad():    
    d = 0
    n = 0
    cnt = 0
    for i in tqdm(range(len(dataset_eval))) :
        clean,noisy,estim = infer(dataset_eval[i],model)
        try : 
            n+= pesq(16000,clean, noisy, "wb")
            d += pesq(16000,clean, estim, "wb")
            cnt+=1
        except NoUtterancesError : 
            continue
    d /=cnt
    n /=cnt
    print(n)
    print(d)